# 01.01 — Introduction to Lagrangian Mechanics
**Notebook:** `control.01_lagrange.01_Intro.ipynb`

This notebook introduces the minimal computational structure for the Lagrangian,
defined as

$$L = T - U,$$

where:
- $T$ is the kinetic energy,
- $U$ is the potential energy.

We implement a minimal object-oriented representation to serve as the basis for  
the full dynamics and control systems workflow in later notebooks.

In [1]:
from dataclasses import dataclass
from typing import Callable

@dataclass
class Lagrangian:
    kinetic_energy: Callable
    potential_energy: Callable

    @property
    def T(self):
        """Return kinetic energy expression."""
        return self.kinetic_energy

    @property
    def U(self):
        """Return potential energy expression."""
        return self.potential_energy

    def __call__(self, q, q_dot):
        """Evaluate L = T - U."""
        return self.T(q, q_dot) - self.U(q)


In [2]:
# Example: simple harmonic oscillator (no damping)
def T_example(q, q_dot, m=1.0):
    return 0.5 * m * q_dot**2

def U_example(q, k=1.0):
    return 0.5 * k * q**2

L = Lagrangian(kinetic_energy=T_example, potential_energy=U_example)

# Test
q = 0.3
q_dot = 1.2
L(q, q_dot)


0.6749999999999999

## 

In [4]:
from typing import Callable, Union
import sympy as sp

NumberLike = Union[float, int, sp.Expr]

class Lagrangian:
    """
    Minimal L(q, q_dot) = T(q, q_dot) - U(q).
    Parameters for T and U are stored separately.
    """
    def __init__(
        self,
        kinetic_energy: Callable[..., NumberLike],
        potential_energy: Callable[..., NumberLike],
        T_params: dict = None,
        U_params: dict = None,
    ) -> None:
        self.kinetic_energy = kinetic_energy
        self.potential_energy = potential_energy
        self.T_params = T_params or {}
        self.U_params = U_params or {}

    def T(self, q: NumberLike, q_dot: NumberLike) -> NumberLike:
        return self.kinetic_energy(q, q_dot, **self.T_params)

    def U(self, q: NumberLike) -> NumberLike:
        return self.potential_energy(q, **self.U_params)

    def __call__(self, q: NumberLike, q_dot: NumberLike) -> NumberLike:
        return self.T(q, q_dot) - self.U(q)


# ------------------------------------------------------------
# Example: simple harmonic oscillator
# ------------------------------------------------------------
def T_example(q, q_dot, m=1.0):
    return 0.5 * m * q_dot**2

def U_example(q, k=1.0):
    return 0.5 * k * q**2

L = Lagrangian(
    kinetic_energy=T_example,
    potential_energy=U_example,
    T_params={"m": 2.0},
    U_params={"k": 5.0},
)

# Test
q_val = 0.3
qdot_val = 1.2

print("L(q, q_dot) =", L(q_val, qdot_val))


L(q, q_dot) = 1.2149999999999999


In [5]:
from typing import Callable, Any, Union
import sympy as sp

NumberLike = Union[float, int, sp.Expr]

class Lagrangian:
    r"""
    Minimal Lagrangian wrapper for an n-DOF system.

    Mathematically:
        L(q, \dot{q}, t) = T(q, \dot{q}, t) - U(q, t)

    Here, `q` and `q_dot` can be:
      - a scalar (1 DOF), or
      - a vector-like object (tuple, list, np.ndarray, SymPy Matrix) for n DOF.

    The dimension n is entirely determined by how you represent `q`.
    """
    def __init__(
        self,
        kinetic_energy: Callable[..., NumberLike],
        potential_energy: Callable[..., NumberLike],
        T_params: dict | None = None,
        U_params: dict | None = None,
    ) -> None:
        self.kinetic_energy = kinetic_energy  # T(q, q_dot, **T_params)
        self.potential_energy = potential_energy  # U(q, **U_params)
        self.T_params = T_params or {}
        self.U_params = U_params or {}

    def T(self, q: Any, q_dot: Any) -> NumberLike:
        """Kinetic energy T(q, q_dot)."""
        return self.kinetic_energy(q, q_dot, **self.T_params)

    def U(self, q: Any) -> NumberLike:
        """Potential energy U(q)."""
        return self.potential_energy(q, **self.U_params)

    def __call__(self, q: Any, q_dot: Any) -> NumberLike:
        """L(q, q_dot) = T(q, q_dot) - U(q)."""
        return self.T(q, q_dot) - self.U(q)


# ------------------------------------------------------------
# 1-DOF EXAMPLE (still valid, just n = 1 here)
# ------------------------------------------------------------
def T_example(q: float, q_dot: float, m: float = 1.0) -> float:
    return 0.5 * m * q_dot**2

def U_example(q: float, k: float = 1.0) -> float:
    return 0.5 * k * q**2

L = Lagrangian(
    kinetic_energy=T_example,
    potential_energy=U_example,
    T_params={"m": 2.0},
    U_params={"k": 5.0},
)

q_val = 0.3
qdot_val = 1.2

print("L(q, q_dot) =", L(q_val, qdot_val))


L(q, q_dot) = 1.2149999999999999


In [8]:
import sympy as sp
from typing import Union, Any
NumberLike = Union[float, int, sp.Expr]

# ------------------------------------------------------------
# 2-DOF Kinetic and Potential Energies
# ------------------------------------------------------------

def T_2dof(q: Any, q_dot: Any, m1: float = 1.0, m2: float = 1.0) -> NumberLike:
    """
    T = 1/2 m1 q1_dot^2  + 1/2 m2 q2_dot^2
    """
    q1_dot = q_dot[0]
    q2_dot = q_dot[1]
    return 0.5*m1*q1_dot**2 + 0.5*m2*q2_dot**2


def U_2dof(q: Any, k1: float = 1.0, k2: float = 1.0, kc: float = 1.0) -> NumberLike:
    """
    U = 1/2 k1 q1^2 + 1/2 k2 q2^2 + 1/2 kc (q2 - q1)^2
    """
    q1 = q[0]
    q2 = q[1]
    return (
        0.5*k1*q1**2 +
        0.5*k2*q2**2 +
        0.5*kc*(q2 - q1)**2
    )

L_2 = Lagrangian(
    kinetic_energy=T_2dof,
    potential_energy=U_2dof,
    T_params={"m1": 2.0, "m2": 1.5},
    U_params={"k1": 4.0, "k2": 3.0, "kc": 2.0},
)

q     = [0.2, -0.1]
q_dot = [0.5, -0.3]

print("L(q, q_dot) =", L_2(q, q_dot))


L(q, q_dot) = 0.13249999999999995


In [9]:
from typing import Callable, Protocol
import sympy as sp

# -------------------------------------------
# Type protocol for a generalized force Q_i
# -------------------------------------------
class GeneralizedForce(Protocol):
    def __call__(self, q, q_dot, **params):
        ...

# -------------------------------------------
# Extended Lagrangian object
# -------------------------------------------
class Lagrangian:
    def __init__(self,
                 kinetic_energy: Callable,
                 potential_energy: Callable,
                 nonconservative_force: GeneralizedForce | None = None):
        self.kinetic_energy = kinetic_energy
        self.potential_energy = potential_energy
        self.nonconservative_force = nonconservative_force

    @property
    def has_Q(self) -> bool:
        return self.nonconservative_force is not None

    def T(self, q, q_dot, **params):
        return self.kinetic_energy(q, q_dot, **params)

    def U(self, q, q_dot, **params):
        return self.potential_energy(q, q_dot, **params)

    def Q(self, q, q_dot, **params):
        if self.nonconservative_force is None:
            return 0
        return self.nonconservative_force(q, q_dot, **params)

    def __call__(self, q, q_dot, **params):
        return self.T(q, q_dot, **params) - self.U(q, q_dot, **params)


In [11]:
# Example energies
def T_example(q, q_dot, m=1.0, **kwargs):
    return 0.5 * m * q_dot**2

def U_example(q, q_dot=0, k=1.0, **kwargs):
    return 0.5 * k * q**2

def Q_damping(q, q_dot, b=0.4, **kwargs):
    return -b * q_dot


# Example non-conservative force
def Q_damping(q, q_dot, b=0.4):
    return -b*q_dot

L = Lagrangian(T_example, U_example, nonconservative_force=Q_damping)

print("L = ", L(0.3, 1.2, m=2, k=5))
print("Q = ", L.Q(0.3, 1.2, b=0.4))


L =  1.2149999999999999
Q =  -0.48
